# import

In [ ]:
from wrappy import util
import glob
import polars as pl

# bitflyer約定履歴取得

In [ ]:
util.bf_get_trades('2023/02/01 00:00:00', '2023/02/10 00:00:00')

# 複数の約定履歴をまとめる

In [ ]:
dir_path = "bitflyer/FX_BTC_JPY/trades/*.csv" # ファイルパス
csv_files = glob.glob(dir_path) # 拡張子が「*csv」だけのファイルを返す
path = [ csv_i for csv_i in csv_files] # フォルダ内のすべてのcsvファイルの名前をリストにする
# フォルダ内のすべてのcsvファイルを読み込みリストにする
df_concat = [(
      pl.read_csv(path_i)
      .with_columns(pl.col("exec_date").str.strptime(pl.Datetime, strict=False))
      ) for path_i in path] 
df_concat = pl.concat(df_concat) # 縦に結合
df_concat = df_concat.unique() # 重複行削除
df_concat.write_csv("2023-01-17_2023-02-24.csv") # csv書き出し

# csvファイル読み込みから1秒足を作るまでワンライナーでやる見本(遅延評価有)

In [ ]:
# 遅延評価有で1秒速くなる
df = (
      pl.scan_csv("bitflyer/FX_BTC_JPY/trades/2023-01-17_2023-02-24.csv")
      .with_columns(pl.col("exec_date").str.strptime(pl.Datetime, strict=False))
      .with_columns(pl.when(pl.col('side') == 'BUY').then(pl.col('size')).otherwise(0).alias('buy_size'))
      .with_columns(pl.when(pl.col('side') == 'SELL').then(pl.col('size')).otherwise(0).alias('sell_size'))
      .groupby_dynamic('exec_date', every='1s')
      .agg([
          pl.col('price').first().alias('open'),
          pl.col('price').max().alias('high'),
          pl.col('price').min().alias('low'),
          pl.col('price').last().alias('close'),
          pl.col('size').sum().alias('volume'),
          pl.col('buy_size').sum().alias('buy_vol'),
          pl.col('sell_size').sum().alias('sell_vol'),
          ])
      .collect()
      )